In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import io
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
#nltk.download () # Download only required or all -> press d, type all, type quit after
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, accuracy_score, f1_score, confusion_matrix, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
train_path = "content/training.csv"
val_path = "content/validation.csv"
test_path = "content/test.csv"

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

Tokenization method to tokenize our datasets

In [3]:
def tokenize(column):
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]  

In [4]:
train_df['tokenized'] = train_df.apply(lambda x: tokenize(x['text']), axis=1)
val_df['tokenized'] = val_df.apply(lambda x: tokenize(x['text']), axis=1)
test_df['tokenized'] = test_df.apply(lambda x: tokenize(x['text']), axis=1)

In [5]:
lmtzr = WordNetLemmatizer()
train_df['lemmatized'] = train_df['tokenized'].apply(
                    lambda lst:[lmtzr.lemmatize(word) for word in lst])
val_df['lemmatized'] = val_df['tokenized'].apply(
                   lambda lst:[lmtzr.lemmatize(word) for word in lst])
test_df['lemmatized'] = test_df['tokenized'].apply(
                   lambda lst:[lmtzr.lemmatize(word) for word in lst])

print(train_df['lemmatized'])
print(val_df['lemmatized'])
print(test_df['lemmatized'])

0                             [i, didnt, feel, humiliated]
1        [i, can, go, from, feeling, so, hopeless, to, ...
2        [im, grabbing, a, minute, to, post, i, feel, g...
3        [i, am, ever, feeling, nostalgic, about, the, ...
4                                [i, am, feeling, grouchy]
                               ...                        
15995    [i, just, had, a, very, brief, time, in, the, ...
15996    [i, am, now, turning, and, i, feel, pathetic, ...
15997                [i, feel, strong, and, good, overall]
15998    [i, feel, like, this, wa, such, a, rude, comme...
15999    [i, know, a, lot, but, i, feel, so, stupid, be...
Name: lemmatized, Length: 16000, dtype: object
0       [im, feeling, quite, sad, and, sorry, for, mys...
1       [i, feel, like, i, am, still, looking, at, a, ...
2                   [i, feel, like, a, faithful, servant]
3               [i, am, just, feeling, cranky, and, blue]
4       [i, can, have, for, a, treat, or, if, i, am, f...
              

Stemming

In [6]:
stemmer = SnowballStemmer("english")
train_df['lemmatized'] = train_df.lemmatized.map(lambda l: [stemmer.stem(word) for word in l])
train_df.lemmatized = train_df.lemmatized.str.join(sep=' ')

val_df['lemmatized'] = val_df.lemmatized.map(lambda l: [stemmer.stem(word) for word in l])
val_df.lemmatized = val_df.lemmatized.str.join(sep=' ')

test_df['lemmatized'] = test_df.lemmatized.map(lambda l: [stemmer.stem(word) for word in l])
test_df.lemmatized = test_df.lemmatized.str.join(sep=' ')

display(train_df["lemmatized"])
display(val_df["lemmatized"])
display(test_df["lemmatized"])

0                                      i didnt feel humili
1        i can go from feel so hopeless to so damn hope...
2              im grab a minut to post i feel greedi wrong
3        i am ever feel nostalg about the fireplac i wi...
4                                        i am feel grouchi
                               ...                        
15995    i just had a veri brief time in the beanbag an...
15996    i am now turn and i feel pathet that i am stil...
15997                        i feel strong and good overal
15998    i feel like this wa such a rude comment and im...
15999    i know a lot but i feel so stupid becaus i can...
Name: lemmatized, Length: 16000, dtype: object

0       im feel quit sad and sorri for myself but ill ...
1       i feel like i am still look at a blank canva b...
2                             i feel like a faith servant
3                          i am just feel cranki and blue
4           i can have for a treat or if i am feel festiv
                              ...                        
1995    im have ssa examin tomorrow in the morn im qui...
1996    i constant worri about their fight against nat...
1997    i feel it import to share this info for those ...
1998    i truli feel that if you are passion enough ab...
1999    i feel like i just wan na buy ani cute make up...
Name: lemmatized, Length: 2000, dtype: object

0       im feel rather rotten so im not veri ambiti ri...
1                   im updat my blog becaus i feel shitti
2       i never make her separ from me becaus i don t ...
3       i left with my bouquet of red and yellow tulip...
4              i wa feel a littl vain when i did this one
                              ...                        
1995    i just keep feel like someon is be unkind to m...
1996    im feel a littl cranki negat after this doctor...
1997    i feel that i am use to my peopl and that give...
1998    im feel more comfort with derbi i feel a thoug...
1999    i feel all weird when i have to meet w peopl i...
Name: lemmatized, Length: 2000, dtype: object

Preprocessing...

In [7]:
cv = CountVectorizer(stop_words='english')
train_ppd_df = cv.fit_transform(train_df["lemmatized"])
val_ppd_df = cv.transform(val_df["lemmatized"])
test_ppd_df = cv.transform(test_df["lemmatized"])


display(train_ppd_df)
display(val_ppd_df)
display(test_ppd_df)

<16000x10082 sparse matrix of type '<class 'numpy.int64'>'
	with 136700 stored elements in Compressed Sparse Row format>

<2000x10082 sparse matrix of type '<class 'numpy.int64'>'
	with 16253 stored elements in Compressed Sparse Row format>

<2000x10082 sparse matrix of type '<class 'numpy.int64'>'
	with 16432 stored elements in Compressed Sparse Row format>

In [8]:
train_labels = np.array(train_df['label'])
val_labels = np.array(val_df['label'])
test_labels = np.array(test_df['label'])

Now training our model with Multinomial Naive Bayes and testing the accuracy with test data

In [9]:
mnb = MultinomialNB()
mnb.fit(train_ppd_df,train_labels)
predictions_NB = mnb.predict(test_ppd_df)

print("Accuracy Score -> ",accuracy_score(predictions_NB, test_labels)*100)
confusion_matrix(y_true=test_labels, y_pred=predictions_NB)

Accuracy Score ->  77.55


array([[528,  38,   2,   8,   5,   0],
       [ 22, 653,   8,   7,   5,   0],
       [ 27,  76,  51,   4,   1,   0],
       [ 58,  30,   0, 180,   7,   0],
       [ 49,  29,   0,   9, 137,   0],
       [ 16,  33,   0,   1,  14,   2]])